In [ ]:
from transformers import BertForMaskedLM, BertTokenizer
import torch
import pandas as pd

In [ ]:
# 데이터 불러오기
df = pd.read_excel(r"file_path")
df.head()

In [ ]:
# 데이터 전처리
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer
import torch

# 한국어 BERT 모델과 토크나이저 로드
bert_model_name = "kykim/bert-kor-base"
bert_model = BertForMaskedLM.from_pretrained(bert_model_name)
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# 감성 분석 함수 정의
def predict_stock_sentiment_bert_korean(headline):
    # 문장 구조를 한국어에 맞게 변경
    sentence = f"{headline}는 LNG 가격에 [MASK] 영향을 줄 것이다."
    input_ids = bert_tokenizer.encode(sentence, return_tensors="pt")
    mask_pos = input_ids[0].tolist().index(bert_tokenizer.mask_token_id)

    with torch.no_grad():
        logits = bert_model(torch.tensor(input_ids))[0]
        predicted_token_id = torch.argmax(logits[0, mask_pos]).item()
        predicted_word = bert_tokenizer.decode(predicted_token_id)
    
    print(predicted_word)
    predicted_word = predicted_word.replace(" ", "")
    print(predicted_word)

    # 긍정/부정 단어 감정 분석 (한국어의 경우에는 '긍정적', '부정적'과 같은 단어를 기준으로 할 수 있습니다.)
    if predicted_word == '긍정적인':
        return 1
    elif predicted_word == '부정적인':
        return -1
    else:
        return 0

In [ ]:
#Test

# 뉴스 헤드라인 예시 (한국어 예시로 변경)
headline_korean = "News_HeadLine"
sentiment_bert_korean = predict_stock_sentiment_bert_korean(headline_korean)

# 결과 출력
headline_korean, sentiment_bert_korean

In [ ]:
# Apply the function to each headline
df["sentiment_bert_korean"] = df["News"].apply(predict_stock_sentiment_bert_korean)
# Extract the month from the date
df['Month'] = df['Date'].dt.to_period('M')
# Group by month and calculate the sum of ChatGPT scores for each month
monthly_scores = df.groupby('Month')["sentiment_bert_korean"].sum().reset_index()

print(monthly_scores)

In [ ]:
monthly_scores.to_excel('monthly_scores.xlsx', index=False)